In [1]:
#pip install -U pandas-profiling;

In [2]:
import pandas_profiling
import pandas as pd
from sklearn import datasets
import numpy as np
import time

In [3]:
data = datasets.load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

In [4]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
#https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/index.html 
report = pandas_profiling.ProfileReport(df)

In [7]:
#report

In [8]:
# lifted from https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype not in [object, bool]:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            '''
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True) 
            '''
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True
            

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [9]:
reduce_mem_usage(df)

Memory usage of properties dataframe is : 0.0503082275390625  MB
******************************
Column:  CRIM
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  ZN
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  INDUS
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  CHAS
dtype before:  float64
dtype after:  uint8
******************************
******************************
Column:  NOX
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  RM
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  AGE
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  DIS
dtype before:  float64
dtype after:  float32
******

(        CRIM    ZN  INDUS  CHAS    NOX     RM        AGE     DIS  RAD  TAX  \
 0    0.00632  18.0   2.31     0  0.538  6.575  65.199997  4.0900    1  296   
 1    0.02731   0.0   7.07     0  0.469  6.421  78.900002  4.9671    2  242   
 2    0.02729   0.0   7.07     0  0.469  7.185  61.099998  4.9671    2  242   
 3    0.03237   0.0   2.18     0  0.458  6.998  45.799999  6.0622    3  222   
 4    0.06905   0.0   2.18     0  0.458  7.147  54.200001  6.0622    3  222   
 ..       ...   ...    ...   ...    ...    ...        ...     ...  ...  ...   
 501  0.06263   0.0  11.93     0  0.573  6.593  69.099998  2.4786    1  273   
 502  0.04527   0.0  11.93     0  0.573  6.120  76.699997  2.2875    1  273   
 503  0.06076   0.0  11.93     0  0.573  6.976  91.000000  2.1675    1  273   
 504  0.10959   0.0  11.93     0  0.573  6.794  89.300003  2.3889    1  273   
 505  0.04741   0.0  11.93     0  0.573  6.030  80.800003  2.5050    1  273   
 
        PTRATIO           B  LSTAT  
 0    15.3000

# Feature Scaling

### Motivation

Why do we apply feature scaling?

 - Preprocessing technique on features to normalize the data into a specific range:- example `[0, 1]`, `[-1, 1]`

 - The ability to visualize data with different ranges at the same time

 - In some ML algorithms the error is calculated using norms (L1, L2 etc.) and the impact of features with a larger range will be higher.

 - In some algorithms, applying on data with smaller ranger is computationally  more efficient. (example Faster conversion for Gradient Descent)

## Implementations

### Min - Max Normalization

Normalizing the data between a specific range using the min & max values. Usaually conversion into `[0,1]` range.
\begin{equation*}
Normalized = \frac{x - min(x)}{max(x) - min(x)}
\end{equation*}

And for the general equation to normalize into `[a,b]` range:

\begin{equation*}
Normalized = a + \frac{(x - min(x))(b - a)}{max(x) - min(x)}
\end{equation*}

In [10]:
# Normal Approach
#%%time 
start_time = time.time()
df_minmax_norm = (df - df.min()) / (df.max() - df.min())
print(f"Execution Time = {time.time() - start_time} using normal approach")
df_minmax_norm.head()


Execution Time = 0.018951892852783203 using normal approach


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000000,0.18,0.067815,0.0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680
1,0.000236,0.00,0.242302,0.0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470
2,0.000236,0.00,0.242302,0.0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466
3,0.000293,0.00,0.063050,0.0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389
4,0.000705,0.00,0.063050,0.0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338


In [11]:
# Lambda Approach
start_time = time.time()
df_minmax_norm = df.apply(lambda x : (x - x.min()) / (x.max() - x.min()))
print(f"Execution Time = {time.time() - start_time} using lambda approach")
df_minmax_norm.head()

Execution Time = 0.012964248657226562 using lambda approach


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000000,0.18,0.067815,0.0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680
1,0.000236,0.00,0.242302,0.0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470
2,0.000236,0.00,0.242302,0.0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466
3,0.000293,0.00,0.063050,0.0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389
4,0.000705,0.00,0.063050,0.0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338


In [12]:
# Skrlearn Approach
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
start_time = time.time()
df_minmax_norm = pd.DataFrame(data = scaler.fit_transform(df), columns = df.columns)
print(f"Execution Time = {time.time() - start_time} using sklearn approach")
df_minmax_norm.head()

Execution Time = 0.007980585098266602 using sklearn approach


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000000,0.18,0.067815,0.0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680
1,0.000236,0.00,0.242302,0.0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470
2,0.000236,0.00,0.242302,0.0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466
3,0.000293,0.00,0.063050,0.0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389
4,0.000705,0.00,0.063050,0.0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338


### Z score Normalization (Standardisation)

Normalizing the data to a standerdized score and the data has a **mean of 0** & **std of 1**.
\begin{equation*}
Normalized = \frac{x - \mu}{\sigma}
\end{equation*}


In [13]:
# Normal Approach
#%%time 
start_time = time.time()
df_zscore_norm = (df - df.mean()) / (df.std())
print(f"Execution Time = {time.time() - start_time} using normal approach")
df_zscore_norm.head()

Execution Time = 0.008978128433227539 using normal approach


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.419367,0.284549,-1.286639,-0.272329,-0.144079,0.413261,-0.119896,0.140075,-0.981871,-0.665949,-1.457581,0.440610,-1.074499
1,-0.416927,-0.487241,-0.592797,-0.272329,-0.739534,0.194081,0.366803,0.556609,-0.867024,-0.986353,-0.302818,0.440610,-0.491952
2,-0.416929,-0.487241,-0.592797,-0.272329,-0.739534,1.281444,-0.265550,0.556609,-0.867024,-0.986353,-0.302818,0.396029,-1.207532
3,-0.416338,-0.487241,-1.305588,-0.272329,-0.834462,1.015296,-0.809088,1.076671,-0.752178,-1.105022,0.112898,0.415746,-1.360171
4,-0.412074,-0.487241,-1.305588,-0.272329,-0.834462,1.227360,-0.510675,1.076671,-0.752178,-1.105022,0.112898,0.440610,-1.025487


In [14]:
def minMaxScaler():
  for col in df.columns:
    df[col] - df[col].min()

In [15]:
# Lambda Approach
start_time = time.time()
df_zscore_norm = df.apply(lambda x : (x - x.mean()) / (x.std()))
print(f"Execution Time = {time.time() - start_time} using lambda approach")
df_zscore_norm.head()

Execution Time = 0.010973215103149414 using lambda approach


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.419367,0.284549,-1.286639,-0.272329,-0.144079,0.413261,-0.119896,0.140075,-0.981871,-0.665949,-1.457581,0.440610,-1.074499
1,-0.416927,-0.487242,-0.592797,-0.272329,-0.739534,0.194081,0.366803,0.556609,-0.867024,-0.986353,-0.302818,0.440610,-0.491952
2,-0.416929,-0.487242,-0.592797,-0.272329,-0.739534,1.281444,-0.265550,0.556609,-0.867024,-0.986353,-0.302818,0.396029,-1.207532
3,-0.416338,-0.487242,-1.305588,-0.272329,-0.834462,1.015296,-0.809088,1.076671,-0.752178,-1.105022,0.112898,0.415746,-1.360171
4,-0.412074,-0.487242,-1.305588,-0.272329,-0.834462,1.227360,-0.510675,1.076671,-0.752178,-1.105022,0.112898,0.440610,-1.025487


In [16]:
# Skrlearn Approach
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
start_time = time.time()
df_zscore_norm = pd.DataFrame(data = scaler.fit_transform(df), columns = df.columns)
print(f"Execution Time = {time.time() - start_time} using sklearn approach")
df_zscore_norm.head()

Execution Time = 0.005985736846923828 using sklearn approach


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.419782,0.284830,-1.287910,-0.272599,-0.144218,0.413672,-0.120014,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867882,-0.987329,-0.303095,0.441052,-0.492439
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867882,-0.987329,-0.303095,0.396427,-1.208727
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228576,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026502


### Normalization by decimal scaling

Normalizing the data by moving the decimal place until the max_value after scaling < 1, max(|x`|) < 1
\begin{equation*}
Normalized = \frac{x}{10^j}
\end{equation*}


In [17]:
start_time = time.time()
max_values = df.abs().max().astype(int)
max_values = max_values.astype(str).apply(lambda x : len(x))
df_decimel_norm = (df) / (10**max_values)
print(f"Execution Time = {time.time() - start_time} using normal approach")
df_decimel_norm.head()

Execution Time = 0.0029916763305664062 using normal approach


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000063,0.018,0.0231,0.0,0.0538,0.6575,0.0652,0.040900,0.01,0.296,0.153,0.39690,0.0498
1,0.000273,0.000,0.0707,0.0,0.0469,0.6421,0.0789,0.049671,0.02,0.242,0.178,0.39690,0.0914
2,0.000273,0.000,0.0707,0.0,0.0469,0.7185,0.0611,0.049671,0.02,0.242,0.178,0.39283,0.0403
3,0.000324,0.000,0.0218,0.0,0.0458,0.6998,0.0458,0.060622,0.03,0.222,0.187,0.39463,0.0294
4,0.000690,0.000,0.0218,0.0,0.0458,0.7147,0.0542,0.060622,0.03,0.222,0.187,0.39690,0.0533


# List Comprehension

## Motivation

Why do we use list comprehensions?

 - Faster than for loops, given the interpreter can optimize patterns more efficiently.

 - Less code to be written

 - An elegent approach for many functionalities 

## Examples

### Basic Usage

In [18]:
nums = [1, 1, 2, 3, 5, 8]

nums_2 = []

for el in nums:
  nums_2.append(el**2)

# Expression, member, iterable
num_list = [el**2 for el in nums]
num_list

[1, 1, 4, 9, 25, 64]

### Using if

In [19]:
num_list = [el / 2 for el in nums if el % 2 == 0] # Even Number
num_list

[1.0, 4.0]

### Using if-else

In [20]:
num_list = [el / 2 if el % 2 == 0 else ((el + 1) / 2) for el in nums]
num_list

[1.0, 1.0, 1.0, 2.0, 3.0, 4.0]

### Nested conditions

In [21]:
num_list = [el / 2 if el % 2 == 0 else ((el + 1) / 2) if el % 2 == 1 else 0 for el in nums]
num_list

[1.0, 1.0, 1.0, 2.0, 3.0, 4.0]

### Set Comprehension

In [22]:
num_list = {el / 2 if el % 2 == 0 else ((el + 1) / 2) for el in nums}
num_list

{1.0, 2.0, 3.0, 4.0}

### Dictionary Comprehension

In [23]:
num_list = {el: el**2 for el in nums}
num_list

{1: 1, 2: 4, 3: 9, 5: 25, 8: 64}

### List Comprehension in pandas

In [24]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.199997,4.0900,1,296,15.300000,396.899994,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.900002,4.9671,2,242,17.799999,396.899994,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.099998,4.9671,2,242,17.799999,392.829987,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.799999,6.0622,3,222,18.700001,394.630005,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.200001,6.0622,3,222,18.700001,396.899994,5.33


In [25]:
df['B_YEAR'] = [int(2020 - el) for el in df['AGE']]

In [26]:
[int(e1 - e2) for e1,e2 in df[['AGE', 'TAX']].values]

[-230,
 -163,
 -180,
 -176,
 -167,
 -163,
 -244,
 -214,
 -211,
 -225,
 -216,
 -228,
 -272,
 -245,
 -222,
 -250,
 -277,
 -225,
 -270,
 -237,
 -208,
 -217,
 -215,
 -207,
 -212,
 -221,
 -216,
 -218,
 -212,
 -219,
 -212,
 -207,
 -225,
 -212,
 -210,
 -210,
 -217,
 -237,
 -248,
 -230,
 -236,
 -230,
 -226,
 -226,
 -193,
 -199,
 -199,
 -147,
 -137,
 -171,
 -197,
 -180,
 -221,
 -221,
 -421,
 -204,
 -277,
 -215,
 -254,
 -236,
 -217,
 -190,
 -216,
 -240,
 -156,
 -319,
 -305,
 -323,
 -308,
 -312,
 -298,
 -287,
 -297,
 -298,
 -392,
 -353,
 -323,
 -352,
 -344,
 -361,
 -247,
 -210,
 -248,
 -234,
 -199,
 -190,
 -201,
 -190,
 -183,
 -206,
 -203,
 -196,
 -216,
 -241,
 -192,
 -218,
 -206,
 -200,
 -239,
 -213,
 -304,
 -312,
 -298,
 -296,
 -294,
 -287,
 -292,
 -298,
 -286,
 -292,
 -329,
 -350,
 -339,
 -336,
 -347,
 -343,
 -359,
 -349,
 -358,
 -366,
 -118,
 -103,
 -95,
 -91,
 -92,
 -99,
 -92,
 -341,
 -338,
 -342,
 -338,
 -339,
 -339,
 -341,
 -338,
 -338,
 -343,
 -338,
 -338,
 -339,
 -343,
 -337,
 -303,
 -30

In [27]:
df[['AGE', 'TAX']].values

array([[ 65.2, 296. ],
       [ 78.9, 242. ],
       [ 61.1, 242. ],
       ...,
       [ 91. , 273. ],
       [ 89.3, 273. ],
       [ 80.8, 273. ]], dtype=float32)

In [28]:
df[['AGE', 'TAX']]

,AGE,TAX
0,65.199997,296
1,78.900002,242
2,61.099998,242
3,45.799999,222
4,54.200001,222
...,...,...
501,69.099998,273
502,76.699997,273
503,91.000000,273
504,89.300003,273


In [29]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,B_YEAR
0,0.00632,18.0,2.31,0,0.538,6.575,65.199997,4.0900,1,296,15.300000,396.899994,4.98,1954
1,0.02731,0.0,7.07,0,0.469,6.421,78.900002,4.9671,2,242,17.799999,396.899994,9.14,1941
2,0.02729,0.0,7.07,0,0.469,7.185,61.099998,4.9671,2,242,17.799999,392.829987,4.03,1958
3,0.03237,0.0,2.18,0,0.458,6.998,45.799999,6.0622,3,222,18.700001,394.630005,2.94,1974
4,0.06905,0.0,2.18,0,0.458,7.147,54.200001,6.0622,3,222,18.700001,396.899994,5.33,1965
